# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'window_low_bound': [1], 'window_high_bound': [6],
                            'first_candle': [0.5], 'second_candle': [0.5]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:28<00:00, 88.11s/it]


# Check local statistics

In [10]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_LinearReg,1,7,0.5,0.5,3,0,2645,1.641025,0.455417,1695.511125,1204.577083,1.4478,1.6265,1.7095,1.6805,1.7967,1.7133,1.7196,1.6799,1.6593,1.6606,1.6307,1.5984,1.5833,1.6028,1.6407,1.6300,1.6286,1.6260,1.6287,1.6261,1.6288,1.6161,1.6234,1.6273,0.34,0.40,0.46,0.47,0.44,0.42,0.45,0.43,0.42,0.43,0.42,0.43,0.45,0.49,0.49,0.51,0.52,0.53,0.49,0.47,0.47,0.46,0.45,0.49
1,Pattern_LinearReg,1,6,0.5,0.5,3,0,2363,1.636179,0.455417,1503.291371,1076.149583,1.4564,1.6039,1.6890,1.6571,1.8022,1.7137,1.7223,1.6807,1.6582,1.6618,1.6292,1.5883,1.5723,1.5966,1.6360,1.6236,1.6215,1.6190,1.6237,1.6243,1.6280,1.6156,1.6199,1.6250,0.34,0.40,0.45,0.47,0.44,0.42,0.46,0.44,0.42,0.45,0.43,0.43,0.44,0.47,0.48,0.50,0.51,0.52,0.49,0.48,0.48,0.47,0.45,0.49
0,Pattern_LinearReg,1,5,0.5,0.5,3,0,2088,1.603621,0.431667,1260.360300,901.320000,1.4249,1.5808,1.6730,1.6302,1.7911,1.6953,1.7025,1.6620,1.6402,1.6429,1.6085,1.5622,1.5331,1.5525,1.5878,1.5796,1.5795,1.5745,1.5789,1.5767,1.5802,1.5718,1.5779,1.5808,0.33,0.39,0.46,0.48,0.43,0.41,0.45,0.43,0.41,0.43,0.42,0.40,0.40,0.43,0.47,0.47,0.48,0.47,0.42,0.41,0.42,0.45,0.43,0.47


# Research parameter influence

In [22]:
param = 'Pattern_first_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_first_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,1.487589,0.422854,660.273914,550.331146,51745
1,0.6,1.446003,0.393219,546.247849,459.464562,46048
2,0.7,1.468667,0.397260,462.588620,370.147344,36771
3,0.8,1.476269,0.409615,337.850515,271.715229,26671
4,0.9,1.456215,0.388792,207.709507,163.309125,16880


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [11]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [1], 'window_high_bound': [6],
                            'first_candle': [0.5], 'second_candle': [0.5]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)